In [10]:

import scipy
#import pytrends
#from pytrends.request import TrendReq
from numpy import random
import matplotlib.pyplot as plt
from scipy import optimize
import sys
import time
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.io.json import json_normalize
import json
from datetime import datetime, timedelta
from county_spending import get_county_spending, spend_county_state_map, us_state_abbrev, abbrev_us_state
from ACS_data import colname_mask
import re
import pickle

pd.set_option('display.max_columns', None)

Can load pickled df at the bottom of notebook

In [2]:
county_spending = get_county_spending()
covid_cases = pd.read_csv('covid_confirmed_usafacts.csv')
ACS_data = pd.read_csv('ACS_disability_by_county.csv', header=1)

/home/hank/Winter2021/Stat6367/ConsultingProject/county_spending.py:34: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  county_json = json_normalize(r.json())


In [3]:
# Whats up with all the missing counties in ACS_data. We need new ACS data anyway, so something to keep in mind
print(county_spending.shape)
print(covid_cases.shape)
print(ACS_data.shape)

(3233, 6)
(3193, 429)
(840, 416)


In [4]:
covid_cases = covid_cases.rename(columns={'County Name':'county_name'})
covid_cases['county_name'] = covid_cases['county_name'].apply(lambda x: x.strip())
covid_cases.head()

# only cumulative
covid_cases = covid_cases[['countyFIPS','county_name','State','StateFIPS','2021-03-21']]

In [5]:
ACS_data = ACS_data.iloc[:,~ACS_data.columns.str.startswith('Margin')]
rename_dict = colname_mask(ACS_data.columns[2:])
ACS_data = ACS_data.rename(columns = rename_dict)
ACS_data = ACS_data.rename(columns = {'Geographic Area Name':'county_name'})
ACS_data['State'] = ACS_data['county_name'].apply(lambda x: x[x.find(',')+2:])
ACS_data['State'] = ACS_data['State'].apply(lambda x: us_state_abbrev[x] if x in us_state_abbrev.keys() else x)
ACS_data['county_name'] = ACS_data['county_name'].apply(lambda x: x[:x.find(',')])

ACS_data.head()

,id,county_name,tot,tot_M,tot_F,tot_W,tot_B,tot_I,tot_A,tot_P,tot_O,tot_T,tot_w,tot_H,tot_0-5,tot_5-17,tot_18-34,tot_35-64,tot_65-74,tot_75+,tot_aud,tot_aud_0-18,tot_aud_0-18_0-5,tot_aud_0-18_5-17,tot_aud_18-64,tot_aud_18-64_18-34,tot_aud_18-64_35-64,tot_aud_65+,tot_aud_65+_65-74,tot_aud_65+_75+,tot_vis,tot_vis_0-18,tot_vis_0-18_0-5,tot_vis_0-18_5-17,tot_vis_18-64,tot_vis_18-64_18-34,tot_vis_18-64_35-64,tot_vis_65+,tot_vis_65+_65-74,tot_vis_65+_75+,tot_cog,tot_cog_0-18,tot_cog_18-64,tot_cog_18-64_18-34,tot_cog_18-64_35-64,tot_cog_65+,tot_cog_65+_65-74,tot_cog_65+_75+,tot_amb,tot_amb_0-18,tot_amb_18-64,tot_amb_18-64_18-34,tot_amb_18-64_35-64,tot_amb_65+,tot_amb_65+_65-74,tot_amb_65+_75+,tot_self,tot_self_0-18,tot_self_18-64,tot_self_18-64_18-34,tot_self_18-64_35-64,tot_self_65+,tot_self_65+_65-74,tot_self_65+_75+,tot_ind,tot_ind_18-64,tot_ind_18-64_18-34,tot_ind_18-64_35-64,tot_ind_65+,tot_ind_65+_65-74,tot_ind_65+_75+,dis,dis_M,dis_F,dis_W,dis_B,dis_I,dis_A,dis_P,dis_O,dis_T,dis_w,dis_H,dis_0-5,dis_5-17,dis_18-34,dis_35-64,dis_65-74,dis_75+,dis_aud,dis_aud_0-18,dis_aud_0-18_0-5,dis_aud_0-18_5-17,dis_aud_18-64,dis_aud_18-64_18-34,dis_aud_18-64_35-64,dis_aud_65+,dis_aud_65+_65-74,dis_aud_65+_75+,dis_vis,dis_vis_0-18,dis_vis_0-18_0-5,dis_vis_0-18_5-17,dis_vis_18-64,dis_vis_18-64_18-34,dis_vis_18-64_35-64,dis_vis_65+,dis_vis_65+_65-74,dis_vis_65+_75+,dis_cog,dis_cog_0-18,dis_cog_18-64,dis_cog_18-64_18-34,dis_cog_18-64_35-64,dis_cog_65+,dis_cog_65+_65-74,dis_cog_65+_75+,dis_amb,dis_amb_0-18,dis_amb_18-64,dis_amb_18-64_18-34,dis_amb_18-64_35-64,dis_amb_65+,dis_amb_65+_65-74,dis_amb_65+_75+,dis_self,dis_self_0-18,dis_self_18-64,dis_self_18-64_18-34,dis_self_18-64_35-64,dis_self_65+,dis_self_65+_65-74,dis_self_65+_75+,dis_ind,dis_ind_18-64,dis_ind_18-64_18-34,dis_ind_18-64_35-64,dis_ind_65+,dis_ind_65+_65-74,dis_ind_65+_75+,%,%_M,%_F,%_W,%_B,%_I,%_A,%_P,%_O,%_T,%_w,%_H,%_0-5,%_5-17,%_18-34,%_35-64,%_65-74,%_75+,%_aud,%_aud_0-18,%_aud_0-18_0-5,%_aud_0-18_5-17,%_aud_18-64,%_aud_18-64_18-34,%_aud_18-64_35-64,%_aud_65+,%_aud_65+_65-74,%_aud_65+_75+,%_vis,%_vis_0-18,%_vis_0-18_0-5,%_vis_0-18_5-17,%_vis_18-64,%_vis_18-64_18-34,%_vis_18-64_35-64,%_vis_65+,%_vis_65+_65-74,%_vis_65+_75+,%_cog,%_cog_0-18,%_cog_18-64,%_cog_18-64_18-34,%_cog_18-64_35-64,%_cog_65+,%_cog_65+_65-74,%_cog_65+_75+,%_amb,%_amb_0-18,%_amb_18-64,%_amb_18-64_18-34,%_amb_18-64_35-64,%_amb_65+,%_amb_65+_65-74,%_amb_65+_75+,%_self,%_self_0-18,%_self_18-64,%_self_18-64_18-34,%_self_18-64_35-64,%_self_65+,%_self_65+_65-74,%_self_65+_75+,%_ind,%_ind_18-64,%_ind_18-64_18-34,%_ind_18-64_35-64,%_ind_65+,%_ind_65+_65-74,%_ind_65+_75+,State
0,0500000US01003,Baldwin County,220911,106961,113950,189325,17668,N,N,N,N,N,183610,N,10616,36210,40659,86588,28682,18156,(X),46826,10616,36210,127247,40659,86588,46838,28682,18156,(X),46826,10616,36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),127247,40659,86588,46838,28682,18156,31901,15573,16328,27623,2050,N,N,N,N,N,27581,N,0,2538,3850,11393,6070,8050,10749,413,0,413,3431,384,3047,6905,2782,4123,5281,626,0,626,1623,650,973,3032,997,2035,10586,2052,5880,2745,3135,2654,739,1915,15885,0,7555,690,6865,8330,3533,4797,6181,553,2534,419,2115,3094,1030,2064,10780,4500,1200,3300,6280,2131,4149,14.4,14.6,14.3,14.6,11.6,N,N,N,N,N,15.0,N,0.0,7.0,9.5,13.2,21.2,44.3,4.9,0.9,0.0,1.1,2.7,0.9,3.5,14.7,9.7,22.7,2.4,1.3,0.0,1.7,1.3,1.6,1.1,6.5,3.5,11.2,5.0,5.7,4.6,6.8,3.6,5.7,2.6,10.5,7.6,0.0,5.9,1.7,7.9,17.8,12.3,26.4,2.9,1.5,2.0,1.0,2.4,6.6,3.6,11.4,6.2,3.5,3.0,3.8,13.4,7.4,22.9,AL
1,0500000US01015,Calhoun County,111075,52442,58633,80403,24662,N,N,N,N,N,79067,N,6699,17635,24227,42369,12017,8128,(X),24334,6699,17635,66596,24227,42369,20145,12017,8128,(X),24334,6699,17635,66596,24227,42369,20145,12017,8128,(X),17635,66596,24227,42369,20145,12017,8128,(X),17635,66596,24227,42369,20145,12017,8128,(X),17635,66596,24227,42369,201

In [6]:
county_spending = county_spending.rename(columns={'display_name':'county_name'})
county_spending['county_name'] = county_spending['county_name'].fillna('Missing')
county_spending['county_name'] = county_spending['county_name'].apply(lambda x: x + ' County')
county_spending = spend_county_state_map(county_spending, ACS_data, covid_cases)
county_spending['State'] = county_spending['State'].apply(lambda x: us_state_abbrev[x] if x in us_state_abbrev.keys() else x)
county_spending.head()


,amount,county_name,shape_code,population,per_capita,award_count,State
0,3.284831e+10,Los Angeles County,06037,10039107.0,3272.03,747836,CA
1,8.909570e+09,Miami-Dade County,12086,2716940.0,3279.27,423598,FL
2,2.713221e+10,Cook County,17031,5150233.0,5268.15,398169,IL
3,1.348097e+10,Harris County,48201,4713325.0,2860.18,280146,TX
4,6.607329e+09,Broward County,12011,1952778.0,3383.55,226534,FL


In [9]:
# Only has counties present in ACS (smallest df at time of writing)
df = ACS_data.merge(county_spending, how='left', on=['county_name', 'State']).merge(covid_cases, how='left', on=['county_name', 'State'])
df.to_pickle('merged_df.pkl')

---------

### Analysis

In [ ]:
df = pd.read_pickle('merged_df.pkl')